In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException

import time
from datetime import datetime
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

In [ ]:
data = pd.DataFrame(columns=['Source','Section','Section URL','Article Title','Article URL','Keywords','Date'])

In [ ]:
import spacy
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

# Load SpaCy's English language model
nlp = spacy.load('en_core_web_sm')

# Function to extract significant words from a single title using POS filtering
def extract_significant_words_from_title(title):
    doc = nlp(title.lower())
    title_words = []
    # Focus on nouns, proper nouns, and possibly adjectives
    for token in doc:
        if token.pos_ in ('NOUN', 'PROPN', 'ADJ') and not token.is_stop:
            title_words.append(token.lemma_)
    return title_words

# Function to extract significant words from a list of titles
def extract_significant_words(titles):
    processed_titles = []  # List to store processed titles
    for title in titles:
        title_words = extract_significant_words_from_title(title)
        processed_titles.append(' '.join(title_words))  # Join and add to processed titles
    return processed_titles

# Function to find common significant words across all titles
def find_common_significant_words(titles):
    processed_titles = extract_significant_words(titles)
    
    # Flatten list of words for all titles
    all_words = ' '.join(processed_titles).split()
    
    # Count the frequency of each word
    word_counts = Counter(all_words)
    
    # Find words that appear in more than one title (common significant words)
    common_words = [word for word, count in word_counts.items() if count > 1]
    
    return common_words

# Example usage
'''
titles =['Brazil’s Supreme Court threatens to suspend X amid ongoing Musk row', "Brazil bans X: Outrage spreads after Elon Musk's social media platform is suspended", 'Brazilian judge orders suspension of X in dispute with Elon Musk', 'Musk’s Starlink Defies Order to Block X in Brazil']


# Extract significant words from each title
processed_titles = extract_significant_words(titles)
print("Processed Titles with Significant Words:")
for title, words in zip(titles, processed_titles):
    print(f"Title: {title}")
    print(f"Significant Words: {words}\n")
    print(words.split(' '))

# Find common significant words across all titles
common_words = find_common_significant_words(titles)
print(f"Common Significant Words Across All Titles: {common_words}")
'''

In [ ]:
def foxnews():
    url = 'https://www.foxnews.com/'

    driver = webdriver.Firefox()
    driver.get(url)
    driver.find_element(By.CLASS_NAME,'js-menu-toggle').click()

    #   Get Sectors
    sector_dict = {}
    sectors = driver.find_elements(By.CLASS_NAME,'nav-title') 
    for i in sectors:
        sector = i.find_element(By.TAG_NAME,'a').get_attribute('aria-label')
        sector_url = i.find_element(By.TAG_NAME,'a').get_attribute('href')
        if sector not in sector_dict:
            sector_dict[sector] = sector_url
        else:
            break
    driver.quit()
    
    #   Into Sector Dicts
    for s in sector_dict:
        driver = webdriver.Firefox()
        driver.get(sector_dict[s])
        print(s,sector_dict[s])
        #   Scroll
        for i in range(8):
            driver.execute_script("window.scrollTo(0, window.pageYOffset + 700);")
            time.sleep(0.3)
        for ele in driver.find_elements(By.XPATH,'//a[@href]'):
            url = ele.get_attribute('href')
            if url.count('-') <= 2 or len(ele.text) <= 2 or 'police-and-law-enforcement' in url or not url.startswith(sector_dict[s]):
                continue
            text = ele.text
            #print(text,url)
            keywords = extract_significant_words_from_title(text)
            date = datetime.today().strftime('%Y-%m-%d')
            data.loc[len(data)] = ['FOX',s,sector_dict[s],text,url,keywords,date]

        driver.quit()

foxnews()

#   No dates in sections page. Either; use today's date, or open article and get from each specific one

In [ ]:
def cnn():
    url = 'https://www.cnn.com/'

    driver = webdriver.Firefox()
    driver.get(url)

    driver.find_element(By.CLASS_NAME,'header__menu-icon-svg').click()

    sector_dict = {}
    sectors = driver.find_elements(By.CLASS_NAME,'subnav__section-link')
    for i in sectors:
        sector = i.text
        sector_url = i.get_attribute('href')
        if 'about' in sector.lower():
            break
        sector_dict[sector] = sector_url

    driver.quit()
    #   Into Sector Dicts
    for s in sector_dict:
        driver = webdriver.Firefox()
        driver.get(sector_dict[s])
        print(s)
        #   Scroll
        for i in range(8):
            driver.execute_script("window.scrollTo(0, window.pageYOffset + 700);")
            time.sleep(0.3)

        for ele in driver.find_elements(By.XPATH,'//a[@href]'):
            text = ele.text
            url = ele.get_attribute('href')
            if len(text) < 10 or url.count('-') < 3 or text.count(' ') < 2 or url in ['',' '] or 'cnn.com/audio/podcasts' in url:
                continue
            #print(ele.text,ele.get_attribute('href'))
            keywords = extract_significant_words_from_title(text)
            date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            data.loc[len(data)] = ['CNN',s,sector_dict[s],text,url,keywords,date]
        driver.quit()
    
cnn()

#   No dates in sections page. Either; use today's date, or open article and get from each specific one

In [ ]:
def get_wapo_sections():
    url = 'https://www.washingtonpost.com'

    driver = webdriver.Firefox()
    driver.get(url)
    driver.find_element(By.XPATH,'//*[@data-testid="sc-header-sections-menu-button"]').click()
    sec = driver.find_element(By.ID,'sc-sections-nav-drawer')
    l = sec.find_elements(By.XPATH, "//*[starts-with(@id, '/')]")
    sector_dict = {}
    actions = ActionChains(driver)
    for i in l:
        dropdown_trigger = i.find_element(By.TAG_NAME,'div')
        # Hover over the element to trigger the dropdown
        actions.move_to_element(dropdown_trigger).perform()
        test = driver.find_elements(By.TAG_NAME,'ul')
        #for t in test[4].find_elements(By.TAG_NAME,'li'):
        print('h: ',len(test))
        t = test[len(test)-1].find_elements(By.TAG_NAME,'li')
        for j in t:
            url = j.find_element(By.TAG_NAME,'a').get_attribute('href')
            txt = j.find_element(By.TAG_NAME,'a').text
            main = i.find_element(By.TAG_NAME,'a').text.replace("+"," ")
            subcat = main + '/' + txt
            print(subcat,url)
            sector_dict[subcat] = url
        
        driver.execute_script("arguments[0].scrollTop += 85;", sec)
        
    driver.quit()
    return sector_dict

def wapo(sector_dict):
    #   Into Sector Dicts
    for s in sector_dict:
        driver = webdriver.Firefox()
        driver.get(sector_dict[s])
        print("~~~~~~~~~")
        print(s)
        #   Scroll
        for i in range(8):
            driver.execute_script("window.scrollTo(0, window.pageYOffset + 700);")
            time.sleep(0.3)
        try:
            region = driver.find_element(By.CSS_SELECTOR,'[data-qa="main"]')
        except:
            driver.quit()
            continue
        print(len(region.find_elements(By.CSS_SELECTOR,'[data-feature-id="homepage/story"]')))
        for ele in region.find_elements(By.CSS_SELECTOR,'[data-feature-id="homepage/story"]'):
            text = ele.text.replace("\n",'')
            url = ele.find_element(By.TAG_NAME,'a').get_attribute('href')
            if len(text) < 3 or url.count('-') < 3 or text.count(' ') < 3 or len(url) < 5:
                continue
            print(text)
            print(url)
            keywords = extract_significant_words_from_title(text)
            date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            data.loc[len(data)] = ['WAPO',s,sector_dict[s],text,url,keywords,date]
        driver.quit()

sector_dict = get_wapo_sections()
wapo(sector_dict)

In [ ]:
#       NYT
def get_nyt():
    driver = webdriver.Firefox()
    driver.get('https://www.nytimes.com/')
    t = driver.find_elements(By.CSS_SELECTOR, '[data-testid^="nav-item-"]')

    sector_dict = {}
    for i in t:
        ele = i.find_element(By.TAG_NAME,'a')
        url = ele.get_attribute('href')
        text = ele.text
        if 'nytimes.com/spotlight/' in url or text in ['',' ','Games','Wirecutter','Cooking']:
            continue
        sector_dict[text] = url

    driver.quit()
    #   Into Sector Dicts
    for s in sector_dict:
        driver = webdriver.Firefox()
        driver.get(sector_dict[s])
        print("~~~~~~~~~")
        print(s)
        for i in range(8):
            driver.execute_script("window.scrollTo(0, window.pageYOffset + 700);")
            time.sleep(0.3)
        try:
            region = driver.find_element(By.CSS_SELECTOR, '[data-testid="asset-stream"]')
        except:
            driver.quit()
            continue
        print('len: ',len(region.find_elements(By.TAG_NAME,'article')))
        for ele in region.find_elements(By.TAG_NAME,'article'):
            lnkreg = ele.find_element(By.TAG_NAME,'a')
            url = lnkreg.get_attribute('href')
            text = lnkreg.text.replace('\n','')

            stop_hls = ['contact us','your ad choices','terms of service','terms of sale','© 2024 the new york times company','skip to main content','skip to site index']
            if 'nytimes.com/' not in url or len(text) < 3 or text.count(' ') < 3 or text.lower() in stop_hls:
                continue

            date = ele.find_element(By.XPATH,'..').find_element(By.CSS_SELECTOR,'[data-testid="todays-date"]').text
            print(date,text,url)
            
            keywords = extract_significant_words_from_title(text) 
            data.loc[len(data)] = ['NYT',s,sector_dict[s],text,url,keywords,date]
        driver.quit()

get_nyt()

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_similar_articles_with_source_names(datasets, source_names):
    """
    Find similar articles across multiple datasets with emphasis on multi-source similarity.
    Avoid repetitions in the output and use source names instead of indices.
    
    Args:
        datasets (list of DataFrames): List of DataFrames where each DataFrame contains articles from a different source.
        source_names (list of str): List of source names corresponding to each DataFrame in datasets.
    
    Returns:
        list of tuples: Each tuple contains (weighted_similarity_score, [title1, title2, ...], [(index1, source1_name), (index2, source2_name), ...])
    """
    # Store all headlines and their embeddings in a dictionary
    embeddings_dict = {}
    
    for idx, df in enumerate(datasets):
        headline_list = df['Article Title'].tolist()
        embeddings = model.encode(headline_list)
        embeddings_dict[idx] = (headline_list, embeddings)

    # List to store aggregated results
    aggregated_results = []
    processed_pairs = set()
    used_articles = set()  # Track used articles

    # Iterate over all headlines in all datasets
    for i in range(len(datasets)):
        headlines1, embeddings1 = embeddings_dict[i]
        
        for k, title1 in enumerate(headlines1):
            if (k, source_names[i]) in used_articles:
                continue  # Skip if already used in a high-scoring match

            # Initialize variables to aggregate similarity scores
            total_similarity_score = 0
            num_sources_matched = 0
            similar_titles = [title1]
            similar_indices = [(k, source_names[i])]
            
            # Compare against all other datasets
            for j in range(len(datasets)):
                if i == j:
                    continue  # Skip comparison with itself

                headlines2, embeddings2 = embeddings_dict[j]
                
                # Compute cosine similarity between the current headline and all headlines in the other dataset
                cosine_matrix = cosine_similarity([embeddings1[k]], embeddings2)[0]
                
                # Find the most similar headline in the other dataset
                max_sim_index = np.argmax(cosine_matrix)
                max_similarity = cosine_matrix[max_sim_index]
                
                # If similarity is above a threshold, consider it a match
                if max_similarity >= 0.52:  # Threshold to consider as a match
                    pair = tuple(sorted([(k, i), (max_sim_index, j)]))  # Sort to avoid ordering issues
                    if pair in processed_pairs or (max_sim_index, source_names[j]) in used_articles:
                        continue  # Skip if this pair has already been processed or article is already used
                    processed_pairs.add(pair)

                    total_similarity_score += max_similarity
                    num_sources_matched += 1
                    similar_titles.append(headlines2[max_sim_index])
                    similar_indices.append((max_sim_index, source_names[j]))
            
            # Calculate weighted similarity score
            weighted_similarity_score = total_similarity_score * (num_sources_matched / len(datasets))
            
            # Store result if there is at least one match (i.e., num_sources_matched > 1)
            if num_sources_matched > 1:  # Ensure at least one other source matches
                aggregated_results.append((weighted_similarity_score, similar_titles, similar_indices))
                # Mark all articles in the current match as used
                used_articles.update(similar_indices)
    
    # Sort results by weighted similarity score in descending order
    aggregated_results.sort(reverse=True, key=lambda x: x[0])

    return aggregated_results

# Example usage:
CNN = data[data['Source'] == 'CNN']
FOX = data[data['Source'] == 'FOX']
WAPO = data[data['Source'] == 'WAPO']
NYT = data[data['Source'] == 'NYT']

datasets = [CNN, FOX, WAPO, NYT]
source_names = ['CNN', 'FOX', 'WAPO', 'NYT']

similar_articles_across_sources = get_similar_articles_with_source_names(datasets, source_names)

# Output results
'''
for score, titles, indices in similar_articles_across_sources:
    print(f"Weighted Score: {score:.4f}, Articles: {titles}, Indices: {indices}")
'''

In [ ]:
#   How to reverse back to 'data' to get url's and such for each similar article
src = similar_articles_across_sources[1][2][1][1]
i = similar_articles_across_sources[1][2][1][0]
print(i,src)
print(similar_articles_across_sources[1][1][1])
data[data['Source'] == src].iloc[i]['Article URL']

In [ ]:
def get_sim_article_df(similar_articles_across_sources):
    similar_articles_df = pd.DataFrame(columns=['Article Headlines','Article URLs','Keywords','Similarity Weights'])
    for i in range(len(similar_articles_across_sources)):
        titles = similar_articles_across_sources[i][1]
        #print(similar_articles_across_sources[i])
            # Extract processed titles
        processed_titles = find_common_significant_words(titles)
        vectorizer = TfidfVectorizer(stop_words='english')
        tfidf_matrix = vectorizer.fit_transform(processed_titles)

        feature_names = vectorizer.get_feature_names_out()
        tfidf_scores = tfidf_matrix.sum(axis=0).A1  # Sum across all documents for global score

        tfidf_ranking = list(zip(feature_names, tfidf_scores))
        tfidf_ranking = sorted(tfidf_ranking, key=lambda x: x[1], reverse=True)

        keywords = [word for word, score in tfidf_ranking if score > 0]
        urls = []
        for k in similar_articles_across_sources[i][2]:
            urls.append(data[data['Source'] == k[1]].iloc[k[0]]['Article URL'])

        #print(keywords)
        #print(urls)
        similar_articles_df.loc[len(similar_articles_df)] = [titles,urls,keywords,similar_articles_across_sources[i][0]]
    return similar_articles_df

similar_articles_df = get_sim_article_df(similar_articles_across_sources)

In [ ]:
similar_articles_df